In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import scipy as ski
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv("Data/HEMODB.csv",sep=';')

In [3]:
data.head()
data.drop(['Unnamed: 13','Unnamed: 14','Unnamed: 15'],axis=1,inplace=True)

In [4]:
#Managing NaN values 
NaN_values=data.isna().any()
columns_with_NaN=data.columns[NaN_values].tolist()
print(columns_with_NaN)

#replace the missing value with the mean 
for to_replace in enumerate(columns_with_NaN):
    #compute the mean of the column where NaN have to be replaced
    mean=data[columns_with_NaN].mean()
    data[columns_with_NaN]=data[columns_with_NaN].replace(np.nan,mean)
print(data)


['Heart rate']
        ID  Brachial SBP  Brachial DBP  Brachial PP     MAP  Heart rate  \
0        1        98.328        73.229       25.098  81.595   85.608000   
1        2        87.134        64.596       22.538  72.109   74.166000   
2        3        85.728        65.301       20.427  72.110   86.010000   
3        4        87.308        68.627       18.681  74.854   82.570492   
4        5        83.572        65.532       18.040  71.545   97.197000   
...    ...           ...           ...          ...     ...         ...   
4013  4014       169.200        59.184      110.020  95.856   90.499000   
4014  4015       149.840        49.120      100.720  82.694   90.460000   
4015  4016       155.220        48.294      106.930  83.937   80.327000   
4016  4017       164.350        62.557      101.800  96.489   93.305000   
4017  4018       145.850        47.502       98.346  80.284   94.579000   

      Carotid-to-femoral PWV  Stroke volume  Ejection fraction  Central SBP  \
0    

In [12]:
#Removal of the unused features, keeping brSBP, brDBP, HR and cfPWV
u_features={'brSBP':data['Brachial SBP'],'brDBP':data['Brachial DBP'],'HR':data['Heart rate'],'cfPWV':data['Carotid-to-femoral PWV']}
used_features=pd.DataFrame(u_features)

#Second set of used features, adding EF
u2_features={'brSBP':data['Brachial SBP'],'brDBP':data['Brachial DBP'],'HR':data['Heart rate'],'cfPWV':data['Carotid-to-femoral PWV'],'EF':data['Ejection fraction']}
used2_features=pd.DataFrame(u2_features)

#Creating csv datafiles
U1_PATH='Data/u1.csv'
U2_PATH='Data/u2.csv'

used_features.to_csv(U1_PATH,sep=',')
used2_features.to_csv(U2_PATH,sep=',')

In [13]:
#Random Forest Regressor
max_depth=[5,10,20]
n_estimators=[500,700,1000]
N=len(max_depth)*len(n_estimators)

#Array to store the end-systolic predictions for each combination of hyperparameters 
esE=np.zeros((len(used_features),N))


In [14]:
i=0
for m in enumerate (max_depth):
    for n in enumerate (n_estimators):
        y=data['End-systolic elastance']
        x=used_features
        regressor=RandomForestRegressor(n[1],max_depth=m[1])
        regressor.fit(used_features,y)
        y_predict=regressor.predict(used_features)
        esE[:,i]=y_predict
        i+=1
        print(y_predict)

[2.76834917e+08 2.78406483e+08 2.76938002e+08 ... 3.39981851e+08
 3.29908255e+08 3.24931453e+08]
[2.76150006e+08 2.79848213e+08 2.76712715e+08 ... 3.38775318e+08
 3.29841314e+08 3.25792544e+08]
[2.76210069e+08 2.79383523e+08 2.77011488e+08 ... 3.39108523e+08
 3.30215827e+08 3.23983824e+08]
[2.77094478e+08 2.78714435e+08 2.72106095e+08 ... 3.53095721e+08
 3.16030515e+08 3.35152898e+08]
[2.76233060e+08 2.78237262e+08 2.72185975e+08 ... 3.53284662e+08
 3.15760884e+08 3.38372081e+08]
[2.76955903e+08 2.77477158e+08 2.71282304e+08 ... 3.52492610e+08
 3.15651720e+08 3.37249824e+08]
[2.91964582e+08 2.68679576e+08 2.46918447e+08 ... 3.66411408e+08
 2.93592671e+08 3.45109417e+08]
[2.91535664e+08 2.69770284e+08 2.46330064e+08 ... 3.67176338e+08
 2.93060161e+08 3.46685789e+08]
[2.90638775e+08 2.68991951e+08 2.48760134e+08 ... 3.67472984e+08
 2.93435221e+08 3.47621209e+08]


In [16]:
RFR_PATH='Data/RFR_Pred.csv'
esE_csv=pd.DataFrame(esE,columns=['(5,500)','(10,700)','(5,1000)','(10,500)','(10,700)','(10,1000)','(20,500)','(20,700)','(20,1000)'])
esE_csv.to_csv(RFR_PATH,sep=',')

In [9]:
#Support Vector Regressor
C=[1,10,100]
gamma=[0.001,0.01,0.1,1]


In [10]:
#Ridge Regression
alpha=[1,10,100,200]

In [11]:
#Gradient Boosting
learning_rate=[0.01,0.05,1]
n_estimators=[100,500,1000,1750]